In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [5]:
# Indicando onde está o arquivo base do Policarpo
planilha = 'data-g'
codigos = pd.read_excel(f'{planilha}.xlsx')

# Removendo linhas duplicadas com base em uma coluna
codigos = codigos.drop_duplicates(subset=['codigo'])
# Excluindo valores nan
codigos = codigos.dropna(subset=['codigo'])
# Convertendo float em int e salvando uma lista com tolist()
codigos = codigos['codigo'].tolist()

# pvs =[359931, 359963, 359743]
len(codigos)

169

In [6]:
# navegador = webdriver.Chrome()
navegador.implicitly_wait(20) # seconds
navegador.get('https://extranet.lopesrio.com.br/')

time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="tLogin"]').send_keys("182.883.817-90") # Gabriel
navegador.find_element(By.XPATH, '/html/body/form/div[3]/div/section/article/fieldset/label[2]/input').send_keys("lopes2023")
navegador.find_element(By.ID, 'btLogin').click()

In [7]:
contador = 0
for codigoAvulso in codigos:
    print(f"Iniciando processo código: {codigoAvulso}")
    navegador.get(f'https://extranet.lopesrio.com.br/AvulsoCadGer.aspx?CODA={codigoAvulso}')
    time.sleep(5)
    # acessar os documentos do imóvel
    navegador.implicitly_wait(20) # seconds
    navegador.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[1]/div[3]/div/div/div/div/div[1]/div/div[1]/ul/li[4]/a').click()

    tabela = navegador.find_elements(By.XPATH, '//*[@id="docs"]/div[3]/div/table/tbody/tr')
    linhas = len(tabela)

    if linhas > 0:
        for i in range(linhas):
            try:
                # excluir arquivos de documentos
                excluir = navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_rptDocs_ctl00_exc')
                excluir.click()
                time.sleep(10)
                alert = navegador.switch_to.alert
                alert.accept()
                alert.accept()
                time.sleep(5)
                linkDocumentacao = navegador.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[1]/div[3]/div/div/div/div/div[1]/div/div[1]/ul/li[4]/a')
                while linkDocumentacao.text != 'Documentação':
                    time.sleep(5)
                linkDocumentacao.click()
            except:
                pass
    
    print(f"Documentos excluídos: {codigoAvulso}")
    # acessar página das imagens
    
    time.sleep(10)
    navegador.get(f'https://extranet.lopesrio.com.br/AvulsoCadGer.aspx?CODA={codigoAvulso}')
    time.sleep(5)
    linkFoto = navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btFotos')
    while linkFoto.text != "Fotos e Imagens":
        time.sleep(5)
    linkFoto.click()
    # verificar se existe fotos
    fotos = navegador.find_elements(By.CLASS_NAME, 'cbp-item')

    if len(fotos) > 1:
        for i in range(len(fotos)):
            if i < 10:
                navegador.implicitly_wait(20) # seconds
                select = Select(navegador.find_element(by=By.ID, value=f"ctl00_ContentPlaceHolder1_rptFotos_ctl0{i}_Status"))
                # select by visible text
                select.select_by_visible_text('REPROVADA')
            else:
                navegador.implicitly_wait(20) # seconds
                select = Select(navegador.find_element(by=By.ID, value=f"ctl00_ContentPlaceHolder1_rptFotos_ctl{i}_Status"))
                # select by visible text
                select.select_by_visible_text('REPROVADA')

    
    navegador.implicitly_wait(20) # seconds
    print(f"Fotos excluídas: {codigoAvulso}")
    time.sleep(10)
    navegador.implicitly_wait(20) # seconds
    navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btSave').click()
    time.sleep(5)
    alert = navegador.switch_to.alert
    alert.accept()

    time.sleep(5)
    navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_lnkVoltar').click()
    time.sleep(10)

    satusVenda = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlStatusVenda"))
    satusVenda.select_by_visible_text('Inativo')

    publicarVenda = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlSite"))
    publicarVenda.select_by_visible_text('NÃO PUBLICAR')

    zapVendas = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlZap"))
    zapVendas.select_by_visible_text('NÃO PUBLICAR')

    zapLocacao = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlZapLoca"))
    zapLocacao.select_by_visible_text('NÃO PUBLICAR')

    IWVendas = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlIW"))
    IWVendas.select_by_visible_text('NÃO PUBLICAR')

    IWLocacao = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlIWLoca"))
    IWLocacao.select_by_visible_text('NÃO PUBLICAR')

    VRVendas = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlVivaReal"))
    VRVendas.select_by_visible_text('NÃO PUBLICAR')

    VRLocacao = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlVivaRealLoca"))
    VRLocacao.select_by_visible_text('NÃO PUBLICAR')

    # alterar o status do imóvel
    time.sleep(10)
    navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btn').click()
    alert = navegador.switch_to.alert
    alert.accept()
    contador += 1
    print(f"Processo finalizado: {codigoAvulso}\nTotal de {contador} concluídos")

Iniciando processo código: BR21601
Documentos excluídos: BR21601
Fotos excluídas: BR21601
Processo finalizado: BR21601
Total de 1 concluídos
Iniciando processo código: BR21960
Documentos excluídos: BR21960
Fotos excluídas: BR21960
Processo finalizado: BR21960
Total de 2 concluídos
Iniciando processo código: BR22003
Documentos excluídos: BR22003
Fotos excluídas: BR22003
Processo finalizado: BR22003
Total de 3 concluídos
Iniciando processo código: BR22779
Documentos excluídos: BR22779
Fotos excluídas: BR22779
Processo finalizado: BR22779
Total de 4 concluídos
Iniciando processo código: BR22850
Documentos excluídos: BR22850
Fotos excluídas: BR22850
Processo finalizado: BR22850
Total de 5 concluídos
Iniciando processo código: BR23137
Documentos excluídos: BR23137
Fotos excluídas: BR23137
Processo finalizado: BR23137
Total de 6 concluídos
Iniciando processo código: BR23421
Documentos excluídos: BR23421
Fotos excluídas: BR23421
Processo finalizado: BR23421
Total de 7 concluídos
Iniciando pro

In [ ]:
def loginExtranet(login, senha):
    """
    Passar cpf no formato de texto com ponto e traço
    """
    # navegador = webdriver.Chrome()
    navegador.implicitly_wait(20) # seconds
    navegador.get('https://extranet.lopesrio.com.br/')

    time.sleep(2)
    navegador.find_element(By.XPATH, '//*[@id="tLogin"]').send_keys(login) # Gabriel
    navegador.find_element(By.XPATH, '/html/body/form/div[3]/div/section/article/fieldset/label[2]/input').send_keys(senha)
    navegador.find_element(By.ID, 'btLogin').click()

In [ ]:
def excluirArquivosAvulso(codigos):
    contador = 0
    try:
        for codigoAvulso in codigos:
        print(f"Iniciando processo código: {codigoAvulso}")
        navegador.get(f'https://extranet.lopesrio.com.br/AvulsoCadGer.aspx?CODA={codigoAvulso}')
        time.sleep(5)
        # acessar os documentos do imóvel
        navegador.implicitly_wait(20) # seconds
        navegador.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[1]/div[3]/div/div/div/div/div[1]/div/div[1]/ul/li[4]/a').click()

        tabela = navegador.find_elements(By.XPATH, '//*[@id="docs"]/div[3]/div/table/tbody/tr')
        linhas = len(tabela)

        if linhas > 0:
            for i in range(linhas):
                try:
                    # excluir arquivos de documentos
                    excluir = navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_rptDocs_ctl00_exc')
                    excluir.click()
                    time.sleep(10)
                    alert = navegador.switch_to.alert
                    alert.accept()
                    alert.accept()
                    time.sleep(5)
                    linkDocumentacao = navegador.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[1]/div[3]/div/div/div/div/div[1]/div/div[1]/ul/li[4]/a')
                    while linkDocumentacao.text != 'Documentação':
                        time.sleep(5)
                    linkDocumentacao.click()
                except:
                    pass
        
        print(f"Documentos excluídos: {codigoAvulso}")
        # acessar página das imagens
        
        time.sleep(10)
        navegador.get(f'https://extranet.lopesrio.com.br/AvulsoCadGer.aspx?CODA={codigoAvulso}')
        time.sleep(5)
        linkFoto = navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btFotos')
        while linkFoto.text != "Fotos e Imagens":
            time.sleep(5)
        linkFoto.click()
        # verificar se existe fotos
        fotos = navegador.find_elements(By.CLASS_NAME, 'cbp-item')

        if len(fotos) > 1:
            for i in range(len(fotos)):
                if i < 10:
                    navegador.implicitly_wait(20) # seconds
                    select = Select(navegador.find_element(by=By.ID, value=f"ctl00_ContentPlaceHolder1_rptFotos_ctl0{i}_Status"))
                    # select by visible text
                    select.select_by_visible_text('REPROVADA')
                else:
                    navegador.implicitly_wait(20) # seconds
                    select = Select(navegador.find_element(by=By.ID, value=f"ctl00_ContentPlaceHolder1_rptFotos_ctl{i}_Status"))
                    # select by visible text
                    select.select_by_visible_text('REPROVADA')

        
        navegador.implicitly_wait(20) # seconds
        print(f"Fotos excluídas: {codigoAvulso}")
        time.sleep(10)
        navegador.implicitly_wait(20) # seconds
        navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btSave').click()
        time.sleep(5)
        alert = navegador.switch_to.alert
        alert.accept()

        time.sleep(5)
        navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_lnkVoltar').click()
        time.sleep(10)

        satusVenda = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlStatusVenda"))
        satusVenda.select_by_visible_text('Inativo')

        publicarVenda = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlSite"))
        publicarVenda.select_by_visible_text('NÃO PUBLICAR')

        zapVendas = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlZap"))
        zapVendas.select_by_visible_text('NÃO PUBLICAR')

        zapLocacao = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlZapLoca"))
        zapLocacao.select_by_visible_text('NÃO PUBLICAR')

        IWVendas = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlIW"))
        IWVendas.select_by_visible_text('NÃO PUBLICAR')

        IWLocacao = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlIWLoca"))
        IWLocacao.select_by_visible_text('NÃO PUBLICAR')

        VRVendas = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlVivaReal"))
        VRVendas.select_by_visible_text('NÃO PUBLICAR')

        VRLocacao = Select(navegador.find_element(by=By.ID, value="ctl00_ContentPlaceHolder1_ddlVivaRealLoca"))
        VRLocacao.select_by_visible_text('NÃO PUBLICAR')

        # alterar o status do imóvel
        time.sleep(10)
        navegador.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btn').click()
        alert = navegador.switch_to.alert
        alert.accept()
        contador += 1
        print(f"Processo finalizado: {codigoAvulso}\nTotal de {contador} concluídos")

        codigos.remove(codigoAvulso)
    except:
        print("Tivemos um imprevisto que foi resolvido.")
    return codigos

In [ ]:
loginExtranet("182.883.817-90", "lopes2023")

for i in len(codigos):
    try:
        excluirArquivosAvulso(codigos)
    except:
        loginExtranet("182.883.817-90", "lopes2023")
    print(i)